In [1]:
from spectralmc.sobol_sampler import SobolSampler, BoundSpec
from spectralmc.gbm import SimulationParams, BlackScholes
import pandas as pd

In [2]:
sp = SimulationParams(
    timesteps=1024,
    network_size=256,
    batches_per_mc_run=2048,
    threads_per_block=256,
    mc_seed=42,
    buffer_size=1,
)

In [3]:
total_size_mb=sp.memory_footprint_bytes()/1024/1024
print(total_size_mb)

4096.0


In [4]:
bs_dimensions = {
    "X0": BoundSpec(lower=0.001, upper=10000),
    "K":  BoundSpec(lower=0.001, upper=20000),
    "T":  BoundSpec(lower=0,     upper=10),
    "r":  BoundSpec(lower=-0.2,  upper=0.2),
    "d":  BoundSpec(lower=-0.2,  upper=0.2),
    "v":  BoundSpec(lower=0,     upper=2.0),
}

In [5]:
ss = SobolSampler(pydantic_class=BlackScholes.Inputs,dimensions=bs_dimensions,seed=43)

In [6]:
bs = BlackScholes(sp=sp)

In [7]:
%%time
samples=ss.sample(64)

CPU times: user 0 ns, sys: 309 μs, total: 309 μs
Wall time: 313 μs


In [12]:
%%time
prices=[bs.price_to_host(input) for input in samples]

CPU times: user 6.21 s, sys: 1.06 ms, total: 6.21 s
Wall time: 6.21 s


In [13]:
bs._normal_gen.get_time_spent_synchronizing()

0.0637979507446289